<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/tokenization-improve/notebooks/Batch_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BATCH processing
- processing all files and saving results to scv

In [0]:
#@title Config
_git_branch = "tokenization-improve" #@param {type:"string"}
GLOBALS__={}

In [0]:
#@title Select sections to run: { run: "auto", vertical-output: true, display-mode: "form" }

#@markdown 1. ### Charters
batch_charters = True #@param {type:"boolean"}
batch_charters_contents = True #@param {type:"boolean"}

#@markdown 2. ### Protocols
batch_protocols = True #@param {type:"boolean"}

#@markdown 3. ### Contracts

batch_contract_contents = True #@param {type:"boolean"}
batch_contract_find_sections = True #@param {type:"boolean"}
batch_contracts = True #@param {type:"boolean"}

In [0]:
''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

   

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  print('❤️ DONE importing Code fro GitHub')

 


In [0]:
 

# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow as tf
import tensorflow_hub as hub


# AZ:-INIT EMBEDDER-----------------------------------------------------------------------------------
def _init_embedder():
  if 'elmo_embedder' in GLOBALS__:
    print('👌 Embedder is already created! ')
    return

  from embedding_tools import ElmoEmbedder
  GLOBALS__['elmo_embedder'] = ElmoEmbedder(module_url = 'https://storage.googleapis.com/az-nlp/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz')
  print('❤️ DONE creating words embedding model')
  return GLOBALS__['elmo_embedder']


def _init_contracts():
  if 'ContractAnlysingContext' in GLOBALS__:
    print('👌 Contracts-related tools are already inited ')
    return

  from contract_parser import ContractAnlysingContext
  GLOBALS__['ContractAnlysingContext'] = ContractAnlysingContext(GLOBALS__['elmo_embedder'], GLOBALS__['renderer'])
  print('❤️ DONE initing Contracts-related tools and models ')


# AZ:- THE CODE----------------------------------------------------------------------------------
def _init_the_code():
  if '_init_the_code' in GLOBALS__:
    print('👌 Code is alredy imported!')
    return

  from renderer import SilentRenderer

  class RendererForBatch(SilentRenderer):
    pass

  GLOBALS__['renderer'] = RendererForBatch()
  GLOBALS__['_init_the_code'] = True

  def read_doc(fn):
    import docx2txt, sys, os
    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()

    return text

  GLOBALS__['read_doc'] = read_doc

  print("❤️ DONE initializing the code")

# AZ:---------------------------------------------------------------------------END OF THE THE CODE, See you later


In [5]:
## do preparation here   
    
#1.
_init_import_code_from_gh()
#2.
_init_embedder()
#3.
_init_the_code()
#4. 
# if batch_charters:
#   _init_charters()
if batch_contracts:
  _init_contracts()

fetching code from GitHub.....tokenization-improve


🦊 GIT revision:
510
* tokenization-improve
Even less verbose

Logging achtungs

Even less verbose


installing antiword...
Reading package lists...
Building dependency tree...
Reading state information...
antiword is already the newest version (0.37-11build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.

installing docx2txt...

❤️ DONE importing Code fro GitHub
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


W0802 10:21:44.282207 140173885994880 deprecation_wrapper.py:119] From nlp_tools/embedding_tools.py:132: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 10:21:45.560985 140173885994880 deprecation_wrapper.py:119] From nlp_tools/embedding_tools.py:143: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0802 10:21:45.564817 140173885994880 deprecation_wrapper.py:119] From nlp_tools/embedding_tools.py:143: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.

W0802 10:21:45.570451 140173885994880 deprecation_wrapper.py:119] From nlp_tools/embedding_tools.py:145: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



❤️ DONE creating words embedding model
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading word cases stats model /content/nlp_tools/vocab/word_cases_stats.pickle
❤️ DONE initializing the code
❤️ DONE initing Contracts-related tools and models 


# BATCH
пакетный процессинг уставов, запись результатов в CSV

## Authenticate on Google and mount Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import glob

def read_documents(filename_prefix):
  filenames = []
  filenames += [file for file in glob.glob(filename_prefix + "*.docx", recursive=True)]
  filenames += [file for file in glob.glob(filename_prefix + "*.doc", recursive=True)]


  texts = {}
  for file in filenames:
    try:
      text = GLOBALS__['read_doc'](file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  return texts

In [0]:
# ------
import csv
def _trim_and_pad(l):
  return (l + [''] * 40)[0:10] # padding: make all linese 10 columns-long

def export_csv(filename, lines, headline=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']):
  with open(f'/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/{filename}', mode='w') as csv_file:
    _writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    _writer.writerow(_trim_and_pad(headline) )
    for l in lines:
      ll = _trim_and_pad(l)
      # print(ll)
      _writer.writerow(ll)

# CHARTERS

In [0]:
from charter_parser import CharterDocumentParser
from charter_patterns import CharterPatternFactory
from embedding_tools import ElmoEmbedder


CH_PF = CharterPatternFactory(GLOBALS__['elmo_embedder'])
CH_CTX = CharterDocumentParser(CH_PF)

### 🏺Micro-self-test

In [9]:
microsample1="слово"
microsample2 = """
Общие положения

1.1. В соответствии с настоящим Договором Жертвователь обязуется безвозмездно передать Получателю денежные средства в размере 30 000 (Тридцать тысяч) рублей в качестве пожертвования.

"""

CH_CTX.analyze_charter(microsample1)
CH_CTX.analyze_charter(microsample2)


❤️ ACCOMPLISHED: 	 0.	 Splitting Document into sections ✂️ 📃 -> 📄📄📄
⚠️ WARNING: - Секция наименования компнании не найдена
⚠️ WARNING: - Попытаемся искать просто в начале документа 🚑
----WARNING!: function make_soft_attention_vector is deprecated
----ERROR: make_soft_attention_vector: too few tokens слово 
----WARNING!: function subdoc is deprecated
❤️ ACCOMPLISHED: 	 1.	 extracting NERs (named entities 🏦 🏨 🏛)
Recent parsing warnings:
		 ⚠️ WARNING: - Секция наименования компнании не найдена
		 ⚠️ WARNING: - Попытаемся искать просто в начале документа 🚑
❤️ ACCOMPLISHED: 	 0.	 Splitting Document into sections ✂️ 📃 -> 📄📄📄
----WARNING!: function rectifyed_sum_by_pattern_prefix is deprecated
❤️ ACCOMPLISHED: 	 1.	 extracting NERs (named entities 🏦 🏨 🏛)


({'attention_vector': array([0.15856819, 0.20308314, 0.44035047, 0.46422809, 0.45892921,
         0.22366183, 0.10593686, 0.08844268, 0.0897323 , 0.05845033,
         0.10999921, 0.21157591, 0.09394148, 0.07869598, 0.04399219,
         0.11458343, 0.05021568, 0.08370542, 0.07552669, 0.05326365,
         0.12868634, 0.12525885, 0.07050131, 0.02920814, 0.10561333,
         0.18129593, 0.06377464, 0.0612906 , 0.14995377, 0.16842198,
         0.46758541, 0.48259509]),
  'name': '',
  'tokens': ['Общие',
   'положения',
   '\n',
   '\n',
   '1.1',
   '.',
   'В',
   'соответствии',
   'с',
   'настоящим',
   'Договором',
   'Жертвователь',
   'обязуется',
   'безвозмездно',
   'передать',
   'Получателю',
   'денежные',
   'средства',
   'в',
   'размере',
   '30000',
   '(',
   'Тридцать',
   'тысяч',
   ')',
   'рублей',
   'в',
   'качестве',
   'пожертвования',
   '.',
   '\n',
   '\n'],
  'type': 'org_unknown',
  'type_name': 'undefined'},
 {})

### 🏺 Test parse single charter

In [0]:
if False:
  CH_CTX.analyze_charter(charters['/content/gdrive/My Drive/GazpromOil/Charters/Устав - ГПН-Транспорт_ГОСА-2018.docx'], verbosity=0)

## Read all the charters

In [11]:
if batch_charters or batch_charters_contents:
  charters_filename_prefix='/content/gdrive/My Drive/GazpromOil/Charters/'
  charters = read_documents(charters_filename_prefix)


good: /content/gdrive/My Drive/GazpromOil/Charters/Устав - ГПН-Транспорт_ГОСА-2018.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/Устав_Рег. продажи авг 2018.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/6.1.1(a) Project Tri-Neft - Sunrise Charter.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/Устав 2.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/ЕЮ Устав.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/МНПЗ Устав.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/ГПН Устав.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/яМессояха Устава.docx
good: /content/gdrive/My Drive/GazpromOil/Charters/МНГ Устав.docx
Unexpected error: (<class 'KeyError'>, KeyError("There is no item named 'word/document.xml' in the archive",), <traceback object at 0x7f7c1a356888>)
good: /content/gdrive/My Drive/GazpromOil/Charters/Устав ГПН-Новосибирск (НБ)_с СД.doc
Unexpected error: (<class 'KeyError'>, KeyError("There is no item named 'word/document.xml'

##  Util/export methods 

In [0]:
from typing import List

from legal_docs import LegalDocument, CharterDocument
from patterns import PatternSearchResult
from renderer import org_level_dict
from structures import OrgStructuralLevel
from transaction_values import ValueConstraint

ggg = {
  'RealEstate': '🐌 Неподвижность',
  'Charity': '🙏 Благ-ость',
  'Lawsuit': '🏦 Судебность разная',
  'Other': '👽не известняк (other)'
}


def _populate_structure( doc: LegalDocument):
  lines=[]

  for n in range(len(doc.structure.headline_indexes) - 1):
    i = doc.structure.headline_indexes[n]
    i_next = doc.structure.headline_indexes[n + 1]

    sline = doc.structure.structure[i]
    sline_next = doc.structure.structure[i_next]

    line = [
      '',#B erase
      '',#C
      '',#D
      str(i),#E
      sline.to_string(doc.tokens_cc),#F
      sline_next.span[0]-sline.span[1],
      sline.line_number,
      sline.minor_number,
      str(sline.number),
      sline.level,
      sline._possible_levels[0],
      sline.span[1]
      
    ]
    
    lines.append(line)
  
  return lines

def _populate_org(orginfo) -> int:
  line = [
    orginfo['name'],
    orginfo['type_name'],
    ''
  ]
  # print(orginfo)
  #   from text_tools import untokenize
  #   worksheet.update_cell(the_row, col , orginfo['name'])
  #   worksheet.update_cell(the_row, col + 1, orginfo['type_name'])
  # #   worksheet.update_cell(the_row, col + 2, orginfo['type'])
  #   worksheet.update_cell(the_row, col + 2, untokenize(orginfo['tokens'])[0:300] )

  return [line]


def _populate_rz(charter):
  lines = [ [charter.filename, '?'] ]
   
  lines += _populate_org(charter.org)

  for level in OrgStructuralLevel:
    constraint_search_results: List[PatternSearchResult] = charter.constraints_by_org_level(level)

    line = [
      '', '',
      org_level_dict[level].upper()
    ]

    lines.append(line)
    # renderered=0
    for sentence in constraint_search_results:
      rr = populate_constraints(sentence)
      lines += rr
    # if renderered==0:
    #   r+=1
    #   _clean(ws, r)
    #   ws.update_acell(f'G{r}', 'пустота пустоты НАЙДЕНА ॐ ओङ्कार 🐼 ом шанти шанти  ॐ' )

    #   r+=1

  return lines


def populate_constraints(sentence: LegalDocument):
  lines = []
  subj_type = sentence.subject_mapping["subj"]

  constraints: List[ValueConstraint] = sentence.constraints
  sname = ggg[subj_type.name]

  line = [
    '', '', " ".join(sentence.tokens[0:200]),
    sname
  ]

  lines.append(line)

  if len(constraints) > 0:
    for pv in constraints:
      c = pv.value
      if c.value is not None and c.sign is not None:
        sign_ = ">" if c.sign > 0 else '<'
        line = [
          '', '', '', '', sign_, c.value, c.currency
        ]
        lines.append(line)

  return lines






## Parse all charters

In [13]:
lines=[]
cnt =0
for fn in charters.keys():
  cnt+=1
  print(cnt, "="*100)
  print(fn)
  CH_CTX.analyze_charter(charters[fn], verbosity=0)
  charter: CharterDocument = CH_CTX.charter
  
  lines+=[[fn]]
  lines += _populate_rz(charter)
  lines+=[['']]
  export_csv('charters_data.csv', lines)



1 ====================================================================================================
/content/gdrive/My Drive/GazpromOil/Charters/Устав - ГПН-Транспорт_ГОСА-2018.docx
❤️ ACCOMPLISHED: 	 0.	 Splitting Document into sections ✂️ 📃 -> 📄📄📄
❤️ ACCOMPLISHED: 	 1.	 extracting NERs (named entities 🏦 🏨 🏛)
2 ====================================================================================================
/content/gdrive/My Drive/GazpromOil/Charters/Устав_Рег. продажи авг 2018.docx
❤️ ACCOMPLISHED: 	 0.	 Splitting Document into sections ✂️ 📃 -> 📄📄📄
❤️ ACCOMPLISHED: 	 1.	 extracting NERs (named entities 🏦 🏨 🏛)
3 ====================================================================================================
/content/gdrive/My Drive/GazpromOil/Charters/6.1.1(a) Project Tri-Neft - Sunrise Charter.docx
❤️ ACCOMPLISHED: 	 0.	 Splitting Document into sections ✂️ 📃 -> 📄📄📄
❤️ ACCOMPLISHED: 	 1.	 extracting NERs (named entities 🏦 🏨 🏛)
4 =============================================

# CONTRACTS

#### Rread all Contracts

In [0]:
if batch_contracts or batch_contract_contents or batch_contract_find_sections:
  contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
  contracts = read_documents(contracts_filename_prefix)
print(f'Total CONTRACTS: {len(contracts)}')

In [0]:
from typing import List

from contract_parser import ContractDocument
from ml_tools import ProbableValue
from structures import ContractSubject

known_subjects_dict = {
  ContractSubject.Charity: '🙏 Благотворительность',
  ContractSubject.RealEstate: "🐌 Сделки с имуществом",
  ContractSubject.Lawsuit: "🏦Судебные споры",
  ContractSubject.Deal: "👽Совершение сделки",
  ContractSubject.Other: "👽Прочее"
}


def render_subj(contract: ContractDocument):
  subjects: List[ProbableValue] = contract.subjects
  confidence = 0
  if len(subjects) > 0:
    sorted_ = [y for y in sorted(subjects, key=lambda x: -x.confidence)]
    subject_kind = sorted_[0].value
    confidence = sorted_[0].confidence
  else:
    subject_kind = ContractSubject.Other

  if subject_kind in known_subjects_dict:
    rendering_name = known_subjects_dict[subject_kind]
  else:
    rendering_name = 'прочее'

  return rendering_name, subject_kind, confidence


def _populate_contract(contract: ContractDocument, filename):
  lines = []
  lines += [[filename]]

  doc = contract
  values = contract.contract_values

  line1=['⚠️ Контрагент 1 не выявлен','⚠️ Форма собственности мутная']
  line2=['⚠️ Контрагент 2 не выявлен','⚠️ Форма собственности мутная']
  
  for tag in contract.agents_tags:
    if tag['kind']=='org.1.name':
      line1[0]=tag['value']
    if tag['kind']=='org.2.name':
      line2[0]=tag['value']
    if tag['kind']=='org.1.type':
      line1[1]=tag['value']
    if tag['kind']=='org.2.type':
      line2[1]=tag['value']

  lines.append(line1)
  lines.append(line2)
  if 'subj' in doc.sections:
    section = doc.sections['subj']
    body = section.body.text[:1000]
    headline = section.subdoc.text[:500]

    quote = headline + "\n" + body
  else:
    quote = '⚠️ Раздел о предмете (любви) не обнаружен'

  rendering_name, subject_kind, confidence = render_subj(doc)
  lines.append(['', '', rendering_name, '', '', '', f'{confidence:.4g}', quote])

  if len(values) > 0:

    for pc in values:
      c = pc.value
      line = ['','', '', c.sign, c.value, c.currency, f'{pc.confidence:.4g}', ' '.join(c.context.tokens)[:500]]
      lines.append(line)

  else:
    lines.append(['','', '', "⚠️ Найдено ничто!"])

  # q+='\n\n'+CTX.get_warings()
  # worksheet.update_cell(r, pricequote_col, q)  

  return lines

#-----------------------

CO_CTX = GLOBALS__['ContractAnlysingContext']
lines = _populate_contract(CO_CTX.contract, filename)
# export_csv('contracts.csv', lines)
export_csv('contract___test.csv', lines, ['', '1', '2', 'sign', 'value', 'currency', 'confidence', 'quote', '8', '9'])


## Parse all Contracts


In [0]:
lines=[]
cnt =0
for fn in contracts.keys() :
  cnt+=1
  print(cnt, "="*100)
  print(fn)
  
  CO_CTX.analyze_contract(contracts[fn])  
  lines += _populate_contract(CO_CTX.contract, fn)

  lines+=[['']]
  export_csv('contracts_data.csv', lines, ['', '1', '2', 'sign', 'value', 'currency', 'confidence', 'quote', '8', '9'])

1 ====================================================================================================
/content/gdrive/My Drive/GazpromOil/Contracts/Договор_ООО Зодчий_25 млн.$.docx
❤️ ACCOMPLISHED: 	 0.	 parsing document 👞 and detecting document high-level structure
